# Semantic Kernel Tool Use Example

ဒီစာရွက်မှာ ChromaDB ကို အသုံးပြုပြီး Retrieval-Augmented Generation (RAG) ကို ပေါင်းစပ်ထားတဲ့ Semantic Kernel-based tool တစ်ခုကို ဖန်တီးတဲ့ code ရဲ့ အကျဉ်းချုပ်နဲ့ ရှင်းလင်းချက်တွေကို ပေးထားပါတယ်။ ဒီဥပမာက ChromaDB collection ထဲက ခရီးသွားစာရွက်စာတမ်းတွေကို ရှာဖွေယူဆောင်နိုင်တဲ့ AI agent တစ်ခုကို ဘယ်လိုတည်ဆောက်ရမယ်ဆိုတာကို ပြသထားပြီး၊ အသုံးပြုသူရဲ့ မေးခွန်းတွေကို semantic ရှာဖွေမှုရလဒ်တွေနဲ့ ပေါင်းစပ်ပြီး အသေးစိတ် ခရီးသွားအကြံပေးချက်တွေကို stream လုပ်ပေးနိုင်ပါတယ်။


## ပတ်ဝန်းကျင်ကို စတင်ခြင်း


SQLite ဗားရှင်းပြင်ဆင်မှု  
သင်အောက်ပါအမှားကိုတွေ့ရှိပါက -  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
သင်၏ notebook အစမှာ ဒီ code block ကို uncomment လုပ်ပါ:  


In [ ]:
# %pip install pysqlite3-binary
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

### ပက်ကေ့ဂျ်များကို တင်သွင်းခြင်း  
အောက်ပါကုဒ်သည် လိုအပ်သော ပက်ကေ့ဂျ်များကို တင်သွင်းပါသည်။


In [ ]:
import json
import os
import chromadb
from typing import Annotated, TYPE_CHECKING

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

if TYPE_CHECKING:
    from chromadb.api.models.Collection import Collection

### ဆင်ဆာမီတစ် ကာနယ်နှင့် AI ဝန်ဆောင်မှု ဖန်တီးခြင်း

ဆင်ဆာမီတစ် ကာနယ်ကို ဖန်တီးပြီး အဆင့်ဆင့် OpenAI chat completion ဝန်ဆောင်မှုဖြင့် ဖွဲ့စည်းထားသည်။ အဆိုပါဝန်ဆောင်မှုကို ကာနယ်တွင် ထည့်သွင်းထားပြီး တုံ့ပြန်မှုများကို ဖန်တီးရန် အသုံးပြုနိုင်ပါသည်။


In [ ]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)


# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### Prompt Plugin ကိုသတ်မှတ်ခြင်း

PromptPlugin သည် retrieval context ကိုအသုံးပြု၍ တိုးတက်သော prompt တစ်ခုကို တည်ဆောက်ရန် function ကိုသတ်မှတ်ထားသော native plugin တစ်ခုဖြစ်သည်။


In [ ]:
class PromptPlugin:

    def __init__(self, collection: "Collection"):
        self.collection = collection

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context."
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide your answer."
        )
    
    @kernel_function(name="retrieve_context", description="Retrieve context from the database.")
    def get_retrieval_context(self, query: str) -> str:
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_entries = []
        if results and results.get("documents") and results["documents"][0]:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_entries.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_entries) if context_entries else "No retrieval context found."

### ရာသီဥတုအချက်အလက် Plugin ကိုသတ်မှတ်ခြင်း

WeatherInfoPlugin သည် ခရီးသွားနေရာအတွက် အပူချိန်အချက်အလက်များပေးစွမ်းသော နက်တစ်ဗီတီဗီ Plugin တစ်ခုဖြစ်သည်။


In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

### သွားရောက်စရာနေရာများအကြောင်းအရာ Plugin ကိုသတ်မှတ်ခြင်း

DestinationsPlugin သည် လူကြိုက်များသော ခရီးသွားနေရာများအကြောင်း အသေးစိတ်အချက်အလက်များပေးစွမ်းသော ဒေသခံ Plugin တစ်ခုဖြစ်သည်။


In [ ]:
class DestinationsPlugin:
    # Destination data store with rich details about popular travel locations
    DESTINATIONS = {
        "maldives": {
            "name": "The Maldives",
            "description": "An archipelago of 26 atolls in the Indian Ocean, known for pristine beaches and overwater bungalows.",
            "best_time": "November to April (dry season)",
            "activities": ["Snorkeling", "Diving", "Island hopping", "Spa retreats", "Underwater dining"],
            "avg_cost": "$400-1200 per night for luxury resorts"
        },
        "swiss alps": {
            "name": "The Swiss Alps",
            "description": "Mountain range spanning across Switzerland with picturesque villages and world-class ski resorts.",
            "best_time": "December to March for skiing, June to September for hiking",
            "activities": ["Skiing", "Snowboarding", "Hiking", "Mountain biking", "Paragliding"],
            "avg_cost": "$250-500 per night for alpine accommodations"
        },
        "safari": {
            "name": "African Safari",
            "description": "Wildlife viewing experiences across various African countries including Kenya, Tanzania, and South Africa.",
            "best_time": "June to October (dry season) for optimal wildlife viewing",
            "activities": ["Game drives", "Walking safaris", "Hot air balloon rides", "Cultural village visits"],
            "avg_cost": "$400-800 per person per day for luxury safari packages"
        },
        "bali": {
            "name": "Bali, Indonesia",
            "description": "Island paradise known for lush rice terraces, beautiful temples, and vibrant culture.",
            "best_time": "April to October (dry season)",
            "activities": ["Surfing", "Temple visits", "Rice terrace trekking", "Yoga retreats", "Beach relaxation"],
            "avg_cost": "$100-500 per night depending on accommodation type"
        },
        "santorini": {
            "name": "Santorini, Greece",
            "description": "Stunning volcanic island with white-washed buildings and blue domes overlooking the Aegean Sea.",
            "best_time": "Late April to early November",
            "activities": ["Sunset watching in Oia", "Wine tasting", "Boat tours", "Beach hopping", "Ancient ruins exploration"],
            "avg_cost": "$200-600 per night for caldera view accommodations"
        }
    }

    @kernel_function(
        name="get_destination_info",
        description="Provides detailed information about specific travel destinations."
    )
    def get_destination_info(self, query: str) -> str:
        # Find which destination is being asked about
        query_lower = query.lower()
        matching_destinations = []

        for key, details in DestinationsPlugin.DESTINATIONS.items():
            if key in query_lower or details["name"].lower() in query_lower:
                matching_destinations.append(details)

        if not matching_destinations:
            return (f"User Query: {query}\n\n"
                    f"I couldn't find specific destination information in our database. "
                    f"Please use the general retrieval system for this query.")

        # Format destination information
        destination_info = "\n\n".join([
            f"Destination: {dest['name']}\n"
            f"Description: {dest['description']}\n"
            f"Best time to visit: {dest['best_time']}\n"
            f"Popular activities: {', '.join(dest['activities'])}\n"
            f"Average cost: {dest['avg_cost']}" for dest in matching_destinations
        ])

        return (f"Destination Information:\n{destination_info}\n\n"
                f"User Query: {query}\n\n"
                "Based on the above destination details, provide a helpful response "
                "that addresses the user's query about this location.")

## ChromaDB ကိုတပ်ဆင်ခြင်း

Retrieval-Augmented Generation ကိုလွယ်ကူစေရန်အတွက်၊ အမြဲတမ်းအသုံးပြုနိုင်သော ChromaDB client ကိုဖန်တီးပြီး `"travel_documents"` ဟုအမည်ပေးထားသော collection ကိုဖန်တီးသည် (သို့မဟုတ် ရှိပြီးသားဖြစ်ပါက ပြန်လည်ရယူသည်)။ ထို့နောက်၊ ဤ collection ကို ခရီးသွားစာရွက်စာတမ်းများနှင့် metadata များဖြင့် ဖြည့်ဆည်းသည်။


In [ ]:
collection = chromadb.PersistentClient(path="./chroma_db").create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True,
)

documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours.",
]

collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin(), WeatherInfoPlugin(), PromptPlugin(collection)],
    name="TravelAgent",
    instructions="Answer travel queries using the provided context. If context is provided, do not say 'I have no context for that.'",
)

### Agent ကို Streaming Chat History ဖြင့် အလုပ်လုပ်စေခြင်း
အဓိက asynchronous loop သည် စကားဝိုင်းအတွက် chat history တစ်ခုကို ဖန်တီးပြီး၊ အသုံးပြုသူ၏ input တစ်ခုစီအတွက် အရင်ဆုံး augmented prompt (system message အနေနဲ့) ကို chat history ထဲထည့်သွင်းပြီး agent သည် retrieval context ကို မြင်နိုင်စေပါသည်။ အသုံးပြုသူ၏ message ကိုလည်း ထည့်သွင်းပြီးနောက်မှာ agent ကို streaming ဖြင့် ခေါ်ယူပါသည်။ ထွက်ရှိလာသော output ကို stream ဖြစ်နေသည့်အတိုင်း print လုပ်ပြသည်။


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()



---

**ဝန်ခံချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူလဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအချော်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့ တာဝန်မယူပါ။
